# 2021/05/06

$\newcommand{\ket}[1]{| #1 \rangle}$

## ショアのアルゴリズムの実装

In [ ]:
# Tested with python 3.7.9, qiskit 0.23.5, numpy 1.20.1
import matplotlib.pyplot as plt
import numpy as np

from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram
from math import gcd
from numpy.random import randint
from fractions import Fraction

### 位数の発見

最初に、繰り返しの位数（周期）を発見するアルゴリズムを見てみます。

ここで見てみるのは、$N$を正の整数としたときの関数$f(x) = a^x \bmod N$の振る舞いです。

In [ ]:
N = 35
a = 3

# プロットするデータを計算する
xvals = np.arange(N)
yvals = [np.mod(a**x, N) for x in xvals]

# matplotlibを使って描画
fig, ax = plt.subplots()
ax.plot(xvals, yvals, linewidth=1, linestyle='dotted', marker='x')
ax.set(xlabel='$x$', ylabel='$%i^x$ mod $%i$' % (a, N),
       title="Example of Periodic Function in Shor's Algorithm")
try: # グラフ上にrをプロット
    r = yvals[1:].index(1) + 1
    plt.annotate(text='', xy=(0,1), xytext=(r,1), arrowprops=dict(arrowstyle='<->'))
    plt.annotate(text='$r=%i$' % r, xy=(r/3,1.5))
except:
    print('Could not find period, check a < N and have no common factors.')

### オラクルの実装

以下では、$N=15$を素因数に分解してみます。やりたいことは、$U\ket{m}=\ket{am \bmod N}$となるユニタリー$U$を$|1\rangle$に$x$回繰り返すことでオラクル$U_f$を実装することです。

練習として、$U\ket{m}=\ket{am \bmod 15}$を実行する関数`U_amod15`を以下に実装してみましょう（`U_amod15`は制御ゲートですが、標的ビットのユニタリー演算に対応する部分を書いてみます）。

In [ ]:
def c_amod15(a, power):
    """mod 15による制御ゲート"""
    if a not in [2,4,7,8,11,13,14]:
        raise ValueError("'a' must be 2,4,7,8,11,13 or 14")

    U = QuantumCircuit(4)

    ##################
    ### EDIT BELOW ###
    ##################
    
    #U.?

    ##################
    ### EDIT ABOVE ###
    ##################

    # 以下で制御ゲートに変換
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

`power`は繰り返しの回数を表します。

**解答例**

```python
    for iteration in range(power):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [4,11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13,14]:
            for q in range(4):
                U.x(q)
```


### 回路全体の実装

測定用ビットとして、8量子ビットを使います。

In [ ]:
# 測定用ビットの数
n_count = 8

a = 7

次に、逆QFTの回路を考えます。この{doc}`実習 <2021_04_22_1>`の問題7を参考にして、QFTの**逆回路**`qft_dagger(n)`を書いてみましょう。引数の$n$は測定用ビットの数`n_count`が入ることに注意します。

In [ ]:
def qft_dagger(n):
    """n量子ビットの逆量子フーリエ変換回路を書く"""
    qc = QuantumCircuit(n)

    ##################
    ### EDIT BELOW ###
    ##################
    
    #qc.?

    ##################
    ### EDIT ABOVE ###
    ##################

    qc.name = "QFT^dagger"
    return qc

**解答例**

```python
    # 最初にSwapsを忘れない!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
```

以上の要素を使って、ショアのアルゴリズムを実装する量子回路を作ります。

In [ ]:
# n_count個の測定用量子ビットと、Uを操作するための4つの作業用量子ビットで量子回路を作る
qc = QuantumCircuit(n_count+4, n_count)

# 測定用量子ビットにHゲートをかけて初期化
qc.h(list(range(n_count)))

# 最後の作業用量子ビットを|1>の状態にする
qc.x(3+n_count)

# 制御Uゲートを適用
for q in range(n_count):
    qc.append(c_amod15(a, 2**q), [q]+[i+n_count for i in range(4)])

# 逆QFTを適用
qc.append(qft_dagger(n_count), list(range(n_count)))

# 回路を測定
qc.measure(list(range(n_count)), list(range(n_count)))
qc.draw('mpl')

シミュレータで実行して、結果を確認してみます。

In [ ]:
backend = Aer.get_backend('qasm_simulator')
results = execute(qc, backend, shots=2048).result()
answer = results.get_counts()

def show_distribution(answer):
    n = len(answer)
    x = [int(key,2) for key in list(answer.keys())]
    y = list(answer.values())

    fig, ax = plt.subplots()
    rect = ax.bar(x,y)

    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{:.3f}'.format(height/sum(y)),
                        xy=(rect.get_x()+rect.get_width()/2, height),xytext=(0,0),
                        textcoords="offset points",ha='center', va='bottom')
    autolabel(rect)
    plt.ylabel('Probabilities')
    plt.show()

show_distribution(answer)

### 計算結果の解析
出力された結果から、位相を求めてみます。

In [ ]:
rows, measured_phases = [], []
for output in answer:
    decimal = int(output, 2)  # 10進数に変換
    phase = decimal/(2**n_count)
    measured_phases.append(phase)
    # これらの値をテーブルの行に追加：
    rows.append(["%i" % (decimal),
                 "%i/%i = %.2f" % (decimal, 2**n_count, phase)])
# 結果を表示
print('Register Output              Phase')
print('----------------------------------')

# 回路を実装できたら、以下のコードをコメントアウトして結果を確認
for i in range(len(rows)):
    print('%15s %18s' % (rows[i][0],rows[i][1]))

得られた位相の情報から、連分数アルゴリズムを使用して$s$と$r$を見つけることができます。Pythonの組み込みの`fractions`(分数)モジュールを使用して、小数を`Fraction`オブジェクトに変換できます。

In [ ]:
rows = []
for phase in measured_phases:
    frac = Fraction(phase).limit_denominator(15)
    rows.append([phase, "%i/%i" % (frac.numerator, frac.denominator), frac.denominator])

# 結果を表示
print('     Phase   Fraction     Guess for r')
print('-------------------------------------')

# 回路を実装できたら、以下のコードをコメントアウトして結果を確認
for i in range(len(rows)):
    print('%10s %10s %15s' % (rows[i][0],rows[i][1],rows[i][2]))

`limit_denominator`メソッドを使って、分母が特定の値（ここでは15）を下回る分数で、最も位相の値に近いものを得ています。

測定された結果のうち、2つ（64と192）が正しい答えである$r=4$を与えたことが分かります。